In [1]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import torch.nn.parallel as nn
from dotenv import load_dotenv
import transformers
import os
import torch

In [2]:

print(torch.__version__) #current version
print(torch.cuda.is_available()) #check if GPU is available

2.2.1+cu121
True


In [3]:
# Check for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")  # Move model and tensors to GPU
    print("Using GPU for computations")
else :
    print("GPU unaviable")

Using GPU for computations


In [4]:
load_dotenv()

True

In [5]:

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [7]:

tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
model = transformers.LlamaForCausalLM.from_pretrained('chaoyi-wu/PMC_LLAMA_7B').to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

C:\Users\RajabiMohammad\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RajabiMohammad\.cache\huggingface\hub\models--chaoyi-wu--PMC_LLAMA_7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
sentence = 'i got stomach ache can you help me' 
batch = tokenizer(
            sentence,
            return_tensors="pt", 
            add_special_tokens=False
        ).to(device)

In [9]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [10]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=100,
    trust_remote_code=True,
    device="cuda" if torch.cuda.is_available() else "cpu"  # Explicit device assignment

)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [11]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "i got headach can you help me"

print(llm_chain.run(question))

C:\Users\RajabiMohammad\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
C:\Users\RajabiMohammad\AppData\Roaming\Python\Python312\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 First, we need to know if the headache is a primary headache or a secondary headache. Second, we need to know if the headache is a migraine or a tension-type headache. Third, we need to know if the headache is a migraine with aura or without aura. Fourth, we need to know if the headache is
